<ol>
    <li>run <pre> python xmlToJson.py (change input folder and output destination)</pre> to convert xml annotations to json</li>
</ol>

In [1]:
import os
import numpy as np
from random import shuffle
import matplotlib.pyplot as plt
import scipy.misc
import json
import random
from PIL import Image

In [2]:
all_images = []
img_path = 'D:/diplomovka_data/data/all_data'

for r, d, f in os.walk(img_path):
    for file in f:
        if file.endswith('.jpg'):
            folder = r.split('\\')[1]
            all_images.append(os.path.join(folder, file))
print(len(all_images))

4939


In [3]:
json_file = './out.json'
with open('out.json') as anno_file:
    anno = json.load(anno_file)

In [4]:
image_paths = {}

for im in anno['images']:
    if im['id'] in image_paths:
        print('duplicita')
    image_paths[im['file_name'].replace('.xml', '.jpg')] = im['id']
    
image_annotations = {}
for a in anno['annotations']:
    im_id = a['image_id']
    if im_id not in image_annotations:
        image_annotations[im_id] = {}
    image_annotations[im_id][a['category_id']]= a['bbox']
print(len(image_annotations))
print(len(image_paths))

1399
1465


In [27]:
round(len(patient_folders) * 0.8)

27

In [30]:
shuffle_patient_images = False
if not shuffle_patient_images:
    patient_folders = np.array(list(set([path.split('\\')[0] for path in list(image_paths.keys())])))
    np.random.shuffle(patient_folders)
    split = round(len(patient_folders) * 0.8)
    train_patients = set(patient_folders[:split])
    test_patients = set(patient_folders[split:])
    print(train_patients, test_patients)

{'5', 'test-5', '23', '1', '9', 'test-1', '33', '8', '26', '24', '11', 'test-4', '25', '16', '18', '3', '6', 'test-2', '22', '21', '28', '12', '7', '2', '19', '20', '13'} {'27', '10', '4', '17', '15', '14', 'test-3'}


In [31]:
noot = []
annots = []
sets = {'train': 0, 'val': 0, 'test': 0}

for imgPath in all_images:
    d = dict()
    d['img_paths'] = imgPath
    d['dataset'] = 'lala'
    
    if shuffle_patient_images: 
        if np.random.rand() > 0.2:
            if np.random.rand() > 0.2:
                d['set'] = 'train'
                sets['train'] += 1
            else:
                d['set'] = 'val'
                sets['val'] += 1
        else:
            d['set'] = 'test' 
            sets['test'] += 1
    else:
        patient_folder =  imgPath.split('\\')[0]
        if patient_folder in train_patients:
            if np.random.rand() > 0.2:
                d['set'] = 'train'
                sets['train'] += 1
            else:
                d['set'] = 'val'
                sets['val'] += 1
        else:
            d['set'] = 'test' 
            sets['test'] += 1
            
    d['img_width'] = 512
    d['img_height'] = 512
    d['objpos'] = [512/2, 512/2]
    d['img_height'] = 512
    d['obj_locations'] = []
    left_kidney_bl,left_kidney_ur,right_kidney_bl,right_kidney_ur = [0,0,0],[0,0,0],[0,0,0],[0,0,0]
    if imgPath in image_paths: #with annotation
        id = image_paths[imgPath]
        if id in image_annotations:
            for category, bbx in image_annotations[id].items():
                if category == 1:
                    left_kidney_bl[0] = bbx[0]
                    left_kidney_bl[1] = bbx[1]
                    left_kidney_ur[0] = bbx[0]+ bbx[2]
                    left_kidney_ur[1] = bbx[1]+bbx[3]
                    left_kidney_bl[2] = 1
                    left_kidney_ur[2] = 1
                if category == 2:
                    right_kidney_bl[0] = bbx[0]
                    right_kidney_bl[1] = bbx[1]
                    right_kidney_ur[0] = bbx[0]+ bbx[2]
                    right_kidney_ur[1] = bbx[1]+bbx[3]
                    right_kidney_bl[2] = 1
                    right_kidney_ur[2] = 1
        else:
            noot.append(id)
    d['obj_locations'].extend([left_kidney_bl, left_kidney_ur, right_kidney_bl, right_kidney_ur])
    d['scale_provided'] = 2
    annots.append(d)
print(sets)

{'train': 3320, 'val': 831, 'test': 788}


In [33]:
3320+831+788

4939

In [32]:
j = json.dumps(annots)
with open('converted_out.json', 'w') as outfile:
    json.dump(j, outfile)